<a href="https://colab.research.google.com/github/michhottinger/DS-Unit-2-Kaggle-Challenge/blob/master/Kaggle_Challenge_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [124]:
import sys
# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Kaggle-Challenge/master/data/'
    !pip install category_encoders==2.*
    !pip install pandas-profiling==2.*

import pandas as pd
from sklearn.model_selection import train_test_split

train = pd.merge(pd.read_csv(DATA_PATH+'waterpumps/train_features.csv'), 
                 pd.read_csv(DATA_PATH+'waterpumps/train_labels.csv'))
test = pd.read_csv(DATA_PATH+'waterpumps/test_features.csv')
sample_submission = pd.read_csv(DATA_PATH+'waterpumps/sample_submission.csv')

# Split train into train & val
train, val = train_test_split(train, train_size=0.80, test_size=0.20, 
                              stratify=train['status_group'], random_state=42)

In [125]:
#need to beat 54% majority classification accuracy
train['status_group'].value_counts(normalize=True)

functional                 0.543077
non functional             0.384238
functional needs repair    0.072685
Name: status_group, dtype: float64

In [0]:
import numpy as np


def clean(X):
  #make a copy
  X = X.copy(deep=True)

  #set almost zeros to zero
  X['latitude'] = X['latitude'].replace(-2e-08, 0)

  #replace zeros with NaNs
  cols_with_zeros = ['longitude', 'latitude', 'construction_year', 'gps_height', 'population']
  for col in cols_with_zeros:
        X[col] = X[col].replace(0, np.nan)
  
  #drop duplicates--going to start with all the data for now
  #X = X.drop(columns=[])

# Drop recorded_by (never varies) and id (always varies, random)
  unusable_variance = ['recorded_by', 'id', 'wpt_name']
  X = X.drop(columns=unusable_variance)

# Convert date_recorded to datetime
  #X['date_recorded'] = pd.to_datetime(X['date_recorded'], infer_datetime_format=True)

# Extract components from date_recorded, then drop the original column
  #X['year_recorded'] = X['date_recorded'].dt.year
    #X['month_recorded'] = X['date_recorded'].dt.month
    #X['day_recorded'] = X['date_recorded'].dt.day
  #X = X.drop(columns='date_recorded')
    # Engineer feature: how many years from construction_year to date_recorded
  #X['years'] = X['year_recorded'] - X['construction_year']
  #X['years_MISSING'] = X['years'].isnull()


  #return the cleaned data
  return X

train = clean(train)
val = clean(val)
test = clean(test)

In [127]:
train.head()

,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,status_group
43360,0.0,2011-07-27,NaN,NaN,NaN,33.542898,-9.174777,0,Lake Nyasa,Mpandapanda,Mbeya,12,4,Rungwe,Kiwira,NaN,True,VWC,K,NaN,NaN,gravity,gravity,gravity,vwc,user-group,never pay,never pay,soft,good,insufficient,insufficient,spring,spring,groundwater,communal standpipe,communal standpipe,functional
7263,500.0,2011-03-23,Rc Church,2049.0,ACRA,34.665760,-9.308548,0,Rufiji,Kitichi,Iringa,11,4,Njombe,Imalinyi,175.0,True,WUA,Tove Mtwango gravity Scheme,True,2008.0,gravity,gravity,gravity,wua,user-group,pay monthly,monthly,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,functional
2486,25.0,2011-03-07,Donor,290.0,Do,38.238568,-6.179919,0,Wami / Ruvu,Kwedigongo,Pwani,6,1,Bagamoyo,Mbwewe,2300.0,True,VWC,NaN,False,2010.0,india mark ii,india mark ii,handpump,vwc,user-group,pay per bucket,per bucket,salty,salty,insufficient,insufficient,shallow well,shallow well,groundwater,hand pump,hand pump,functional
313,0.0,2011-07-31,Government Of Tanzania,NaN,DWE,30.716727,-1.289055,0,Lake Victoria,Kihanga,Kagera,18,1,Karagwe,Isingiro,NaN,True,NaN,NaN,True,NaN,other,other,other,vwc,user-group,never pay,never pay,soft,good,enough,enough,shallow well,shallow well,groundwater,other,other,non functional
52726,0.0,2011-03-10,Water,NaN,Gove,35.389331,-6.399942,0,Internal,Mtakuj,Dodoma,1,6,Bahi,Nondwa,NaN,True,VWC,Zeje,True,NaN,mono,mono,motorpump,vwc,user-group,pay per bucket,per bucket,soft,good,enough,enough,machine dbh,borehole,groundwater,communal standpipe,communal standpipe,functional


In [0]:
#select features further

target = 'status_group'

#get a dataframe with all train columns, drop the target
#features = train.drop(columns=[target])--do this in one step below
#FOR ORDINAL ENCODING, I WILL START WILL ALL MY FEATURES--NO NEED TO DROP HIGH CARDIS

#get a list of the numeric features
#numeric_features = train_features.select_dtypes(include='number').columns.tolist()

#get a list of the nonnumerical and limit cardinality
#cardi = train_features.select_dtypes(exclude='number').nunique()

#list the categorical features with cardinality <=30
#cat_features = cardi[cardi <=30].index.tolist()

#combine numerics and nonnumerics
#features = numeric_features + cat_features


In [0]:
#now we can arrange this data
X_train = train.drop(columns=target)
y_train = train[target]
X_val = val.drop(columns=target)
y_val = val[target]
X_test = test

In [130]:
import category_encoders as ce
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler


#explicitly require this experimental feature
#from sklearn.experimental import enable_iterative_imputer

# now you can import normally from sklearn.impute
#from sklearn.impute import IterativeImputer

pipeline = make_pipeline(
    ce.OrdinalEncoder(drop_invariant=True), 
    SimpleImputer(strategy='mean'), 
    RandomForestClassifier(n_estimators=110, random_state=42, n_jobs=-1, min_samples_leaf=2, criterion="entropy")
)

#fit on train
pipeline.fit(X_train, y_train)

#score on train, val
print('Train Accuracy', pipeline.score(X_train, y_train))
print('Validation Accuracy', pipeline.score(X_val, y_val))


Train Accuracy 0.943939393939394
Validation Accuracy 0.8168350168350168


first attempt with years features eng: Validation Accuracy 0.810016835016835 \n
second attempt with out years: Validation Accuracy 0.8105218855218855 
Third attempt with iterative impute at 10: Validation Accuracy 0.807996632996633
fourth attempt with iterative impute at 40: same
Changed the iterative imp back to simple impute and updated classier stuff: entropy(0.8164141414141414) instead of gini(0.8138888888888889)
now drop a few features just to see if we can get better than entropy:nope, more the marrier
Now n_est was changed to n_est = 110: Validation Accuracy 0.8168350168350168 (n_estimators=110, random_state=42, n_jobs=-1, min_samples_leaf=2, criterion="entropy")

In [0]:
#y_pred = pipeline.predict(X_test)
#submission = sample_submission.copy()
#submission['status_group'] = y_pred
#submission.to_csv('michelle-hottinger-submission2.csv', index=False)

In [0]:
#from google.colab import files
#files.download('michelle-hottinger-submission2.csv')